#**Llama 2**

Llama 2 es una colección de modelos de texto generativo preentrenados y afinados, que van desde los 7 mil millones hasta los 70 mil millones de parámetros, diseñados para casos de uso de diálogo.

Supera a los modelos de chat de código abierto en la mayoría de los benchmarks y está a la par de los modelos de código cerrado populares en las evaluaciones humanas de utilidad y seguridad.

Llama 2 13B-chat: https://huggingface.co/meta-llama/Llama-2-13b-chat

El objetivo de llama.cpp es ejecutar el modelo LLaMA con cuantificación de enteros de 4 bits. Es una implementación simple de C/C++ optimizada para las arquitecturas Apple silicon y x86, que admite varias bibliotecas de cuantificación de enteros y BLAS. Originalmente un ejemplo de chat web, ahora sirve como un patio de recreo de desarrollo para las características de la biblioteca ggml.


GGML, una biblioteca C para aprendizaje automático, facilita la distribución de grandes modelos de lenguaje (LLM). Utiliza la cuantificación para permitir la ejecución eficiente de LLM en hardware de consumo. Los archivos GGML contienen datos codificados en binario, que incluyen el número de versión, los hiperparámetros, el vocabulario y los pesos. El vocabulario comprende tokens para la generación de lenguaje, mientras que los pesos determinan el tamaño del LLM. La cuantificación reduce la precisión para optimizar el uso de recursos.

#  Modelos cuantizados de la comunidad Hugging Face

La comunidad Hugging Face proporciona modelos cuantificados, que nos permiten utilizar el modelo de manera eficiente y efectiva en la GPU T4. Es importante consultar fuentes confiables antes de usar cualquier modelo.

Hay varias variaciones disponibles, pero las que nos interesan se basan en la biblioteca GGLM.

Podemos ver las diferentes variaciones que tiene Llama-2-13B-GGML en el siguiente enlace: https://huggingface.co/models?search=llama%202%20ggml

En este caso, usaremos el modelo llamado Llama-2-13B-chat-GGML, que se puede encontrar en el siguiente enlace: https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML

#**Step 1: Instala todas las librerías necesarias**

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.9 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.8/807.8 kB 5.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 47.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 13.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 7.7 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-ifaczk0u/overlay/local/bin
    changing mode of /tmp/pip-build-env-ifaczk0u/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-ifaczk0u/overlay/local

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

#**Step 2: Importa todas las librerías**

In [ ]:
from huggingface_hub import hf_hub_download


In [ ]:
from llama_cpp import Llama


#**Step 3: Descarga el modelo**

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

#**Step 4: Carga el modelo**

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

#**Step 5: Crear un Prompt Template**

In [ ]:
prompt = "Escribe una función en python que me de los n primeros números primos"
prompt_template=f'''SYSTEM: Eres una asistente personal que busca ayudar lo mejor posible.

USER: {prompt}

ASSISTANT:
'''

#**Step 6: Generar las respuestas**

In [ ]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

Llama.generate: prefix-match hit


In [ ]:
print(response["choices"][0]["text"])

SYSTEM: Eres una asistente personal que busca ayudar lo mejor posible.

USER: Escribe una función en python que me de los n primeros números primos

ASSISTANT:

¡Claro! Aquí te dejo un ejemplo de cómo podrías hacerlo:
```
def first_n_primes(n):
    # Función que devuelve una lista con los primeras n números primos
    
    # Definimos una función recursiva para encontrar los números primos
    def is_prime(x):
        if x <= 1:
            return False
        for i in range(2, int(x ** 0.5) + 1):
            if x % i == 0:
                return False
        return True
    
    # Creamos una lista vacía para almacenar los números primos
    prime_list = []
    
    # Iteramos sobre el rango [2, n] para encontrar los números primos
    for i in range(2, n + 1):
        if is_prime(i):
            prime_list.append(i)
    
    return prime_list[:n]
```
¡Eso es todo! La función `first_n_primes` toma


In [ ]:
def first_n_primes(n):
    # Función que devuelve una lista con los primeras n números primos

    # Definimos una función recursiva para encontrar los números primos
    def is_prime(x):
        if x <= 1:
            return False
        for i in range(2, int(x ** 0.5) + 1):
            if x % i == 0:
                return False
        return True

    # Creamos una lista vacía para almacenar los números primos
    prime_list = []

    # Iteramos sobre el rango [2, n] para encontrar los números primos
    for i in range(2, n + 1):
        if is_prime(i):
            prime_list.append(i)

    return prime_list[:n]


a = first_n_primes(20)
a

[2, 3, 5, 7, 11, 13, 17, 19]

In [ ]:
prompt = "Manda un saludo a los seguidores de AlexFocus!"
prompt_template=f'''SYSTEM: Eres una asistente personal que busca ayudar lo mejor posible.

USER: {prompt}

ASSISTANT:
'''
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)
print(response["choices"][0]["text"])

Llama.generate: prefix-match hit


SYSTEM: Eres una asistente personal que busca ayudar lo mejor posible.

USER: Manda un saludo a los seguidores de AlexFocus!

ASSISTANT:
¡Hola a todos mis amigos y seguidores de AlexFocus! Espero que estén teniendo un día increíble. ¡Estoy aquí para ayudar en lo que necesiten! ¿Hay algún tema o pregunta que quieran platicar? 😊
